In [2]:
import boto3
from sagemaker import get_execution_role
from zipfile import ZipFile
import os 

# Get the SageMaker execution role
# sagemaker need permission to write lambda functions

# get the execution role for this notebook
#role = get_execution_role()

# Create a Lambda client using Boto3
client = boto3.client('lambda')



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
role

In [ ]:
# Function to create Lambda function from a Python file
def create_lambda_function(function_name, file_name, role_arn, timeout = 10):
    """
    """
    # Zip the lambda_function.py file
    with ZipFile(f'{file_name}.zip', 'w') as z:
        z.write(f'{file_name}.py')
        
    # check that .zip file was created
    assert os.path.exists(f'{file_name}.zip'), f" File not created {file_name}.zip"
    
    # Read the zipped file content
    with open(f'{file_name}.zip', 'rb') as f:
        b_code = f.read()
        
    # Create the Lambda function
    response = client.create_function(
        FunctionName=function_name,
        Runtime='python3.8',
        Handler=f'{file_name}.lambda_handler',  # lambda handler entry point
        Code={'ZipFile': b_code},
        Description=f'{function_name} for image classification workflow',
        Timeout=timeout,
        MemorySize=1024,
        Publish=True,
        PackageType='Zip',
        # role with full access to lambda (write functions included)
        # also S3 full access
        Role="arn:aws:iam::676719617777:role/lambda_full_access"
    )
    
    print(f'Created Lambda function: {function_name}')
    return response


In [ ]:


# Create 3 Lambda functions
create_lambda_function('serializeImageData', 'serialize_image_data', role, 10 )
# give more time to the classification image
create_lambda_function('classifyImage', 'classify_image', role, 60)
create_lambda_function('filterInferences', 'filter_inferences', role, 10)


### Test Lambda Functions
- call the functions with a payload, a json file

### Step 1: Invoke serializeImageData Lambda and Read the JSON Response

In [ ]:
bucket = 'sagemaker-us-east-1-676719617777'
image = "test/bicycle_s_000513.png"

In [15]:
%%bash

# Invoke the `serializeImageData` Lambda function
aws lambda invoke \
  --function-name serializeImageData \
  --payload '{"s3_bucket": "sagemaker-us-east-1-676719617777", "s3_key": "test/bicycle_s_000513.png"}' \
  serializeImageData_response.json

# View the result
cat serializeImageData_response.json


{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}
{"statusCode": 200, "body": {"image_data": "iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAYAAABzenr0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8fJSN1AAAACXBIWXMAAA9hAAAPYQGoP6dpAAAKhUlEQVR4nF2X2ZNd1XXGf3s65873drd6UKtb6mYQCDmAIBSJ4yR2DLGLUDw5j37MH5PKn5HKSyqp4ErhymSDKSCAFUYBcgvUaqmn24PufM895+y9Vx6unIesp/2011r7W+v7vq3+7h8+lDRJsNYiKuIVJLU2uCoBhXiBIiefnNE/v4eSANHQWH6cUFsjIeCs4AFlDYl2GG0oJVICjgJTTBhNZthKjYjCB4VEAQV2NO6TO0e9XkepiBfAVvDRYhBMds6dzz8kDLpYKVhaqvDNpzf58Rs/xzeqnOUGrxI8EZFAkWpMmVMpp1TGfYaTEaMYmRaetUuXQVti0IgIIoKttDcxWkM6fwWnNGiNDgofC7y2NFZWGYmimJSMVcrmS3/FKJSszG6x0rzIpBCmeYZ2FhMsnXod7/t8cPMtxtLg2p/8NRerDXIv+ChoDSKC9x7rkhoxBlCOiAUBFSJIQCKUqkHn8g2a6wqCgA5k2jDIu7TiHuvqNmflIuOZw2Y9nM1ZXlnBJx2u3PgZvrmEcSl5iEQgiODDvABRBmuM4KxGawEpEYlEgRACiOC0wntFWUYEQSQSrMGYDoXMyGaHeLfJtL6Eczle1zgtBtRNQW2hQV+llCKgFQoh+kDEgJrDYJVSgPD78D7gvUdEUEScUSg0BghRQBQugNEpOR2OB7+jXn9ArurksoT1BudyyphB9KgYCb+/XIFSBhEAQWuNBYhRgEiM

### Step 2: Use the Base64 Image Data from the JSON Response for the classifyImage Function
Now, extract the base64 image data from the serializeImageData_response.json file and use it as the input for the next Lambda function:

In [17]:
%%bash

# Extract the base64 image data (replace base64_image_data_here with actual value from Step 1)
base64_image_data=$(cat serializeImageData_response.json | jq -r '.body.image_data')

# Pass the base64 image data to classifyImage Lambda function
aws lambda invoke \
  --function-name classifyImage \
  --payload "{\"body\": {\"image_data\": \"$base64_image_data\"}}" \
  classifyImage_response.json

# View the result of classifyImage
cat classifyImage_response.json


bash: line 3: jq: command not found


{
    "StatusCode": 200,
    "FunctionError": "Unhandled",
    "ExecutedVersion": "$LATEST"
}
{"errorMessage": "Unable to import module 'classify_image': No module named 'sagemaker'", "errorType": "Runtime.ImportModuleError", "stackTrace": []}